In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install utm


  Preparing metadata (setup.py) ... done
  Created wheel for utm: filename=utm-0.8.0-py3-none-any.whl size=8536 sha256=d381e2527637cb0baf102d5c8684e28b80238dbf563066af377576bf27c324f9
  Stored in directory: /root/.cache/pip/wheels/b0/88/9b/1968d3a5933864f7d6312d8217027dd4d52b9647f678c67023
Successfully built utm


In [ ]:
import cv2
import numpy as np
import utm

# Define landmarks (latitude, longitude) and corresponding pixel coordinates
landmarks_geo = [
    (21.30323333, 72.95387778),
    (21.30486667, 72.95492222),
    (21.30457222, 72.95447778),
    (21.30312222, 72.95451111),
    (21.30548333, 72.95526944),
    (21.30383056, 72.95407778),
    (21.30605556, 72.95398611)
]

# Convert geographical coordinates to UTM
landmarks_utm = np.array([utm.from_latlon(lat, lon)[:2] for lat, lon in landmarks_geo], dtype=np.float32)

# Corresponding pixel coordinates in the first frame
initial_pixel_points = np.array([
    [1843, 759],
    [674, 316],
    [906, 549],
    [1854, 367],
    [231, 169],
    [1439, 711],
    [17, 1020]
], dtype=np.float32)

# Compute the homography matrix
homography_matrix, status = cv2.findHomography(initial_pixel_points, landmarks_utm, method=cv2.RANSAC)

print("Homography Matrix:")
print(homography_matrix)

video_path = "/content/drive/MyDrive/Kamrej 2.mp4"
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter('/content/output_geotagged.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process each pixel point
    for x in initial_pixel_points:
        pixel_point = np.array([x], dtype=np.float32).reshape(-1, 1, 2)
        utm_point = cv2.perspectiveTransform(pixel_point, homography_matrix)
        utm_x, utm_y = utm_point[0][0]

        latitude, longitude = utm.to_latlon(utm_x, utm_y, 43, 'Q')

        center = (int(x[0]), int(x[1]))
        cv2.circle(frame, center, 5, (0, 255, 0), -1)

        text_position = (int(x[0]) + 10, int(x[1]) - 10)
        cv2.putText(frame, f"{latitude:.6f}, {longitude:.6f}", text_position,
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

print("Geotagging complete! Video saved as /content/output_geotagged.mp4")


Homography Matrix:
[[-1.51364659e+00 -6.26378824e+00  2.87929897e+05]
 [-1.23062767e+01 -4.99425190e+01  2.35736371e+06]
 [-5.15176897e-06 -2.12004961e-05  1.00000000e+00]]
Geotagging complete! Video saved as /content/output_geotagged.mp4


In [ ]:
!pip install utm
import cv2
import numpy as np
import utm

# Define landmarks (latitude, longitude) and corresponding pixel coordinates
landmarks_geo = [
    (21.30323333, 72.95387778),
    (21.30486667, 72.95492222),
    (21.30457222, 72.95447778),
    (21.30312222, 72.95451111),
    (21.30548333, 72.95526944),
    (21.30383056, 72.95407778),
    (21.30605556, 72.95398611)
]

# Convert geographical coordinates to UTM
landmarks_utm = np.array([utm.from_latlon(lat, lon)[:2] for lat, lon in landmarks_geo], dtype=np.float32)

# Corresponding pixel coordinates in the first frame
initial_pixel_points = np.array([
    [1843, 759],
    [674, 316],
    [906, 549],
    [1854, 367],
    [231, 169],
    [1439, 711],
    [17, 1020]
], dtype=np.float32)

# Compute the homography matrix
homography_matrix, status = cv2.findHomography(initial_pixel_points, landmarks_utm, method=cv2.RANSAC)

print("Homography Matrix:")
print(homography_matrix)

video_path = "/content/drive/MyDrive/Kamrej 2.mp4"  # Replace with your video path
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter('/content/output_geotagged.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process each pixel point
    for x in initial_pixel_points:
        pixel_point = np.array([x], dtype=np.float32).reshape(-1, 1, 2)
        utm_point = cv2.perspectiveTransform(pixel_point, homography_matrix)
        utm_x, utm_y = utm_point[0][0]

        latitude, longitude = utm.to_latlon(utm_x, utm_y, 43, 'Q')

        center = (int(x[0]), int(x[1]))
        cv2.circle(frame, center, 5, (0, 255, 0), -1)

        text_position = (int(x[0]) + 10, int(x[1]) - 10)
        cv2.putText(frame, f"{latitude:.6f}, {longitude:.6f}", text_position,
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    out.write(frame)

    # Save the first frame for later use
    if 'saved_frame' not in locals():
        saved_frame = frame.copy()

cap.release()
out.release()
cv2.destroyAllWindows()

print("Geotagging complete! Video saved as /content/output_geotagged_2.mp4")

# Now use the saved frame to create the image file
cv2.imwrite("/content/geotagged_frame.jpg", saved_frame)

Homography Matrix:
[[-1.51364659e+00 -6.26378824e+00  2.87929897e+05]
 [-1.23062767e+01 -4.99425190e+01  2.35736371e+06]
 [-5.15176897e-06 -2.12004961e-05  1.00000000e+00]]
Geotagging complete! Video saved as /content/output_geotagged_2.mp4


True

In [ ]:
from google.colab import files
files.download("/content/output_geotagged.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>